In [1]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import tensorflow as tf

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math

## Parameters

In [ ]:
NSAMPLE = 2500

In [ ]:
from aml_dl.mdn.training.config import network_params_inv

In [ ]:
from aml_dl.mdn.model.mdn_push_inv_model import MDNPushInverseModel

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
inverse_model = MDNPushInverseModel(sess=sess, network_params=network_params_inv)
inverse_model.init_model()

In [ ]:
network_params_inv['dim_input'] = 1
network_params_inv['dim_output'] = 1
network_params_inv['k_mixtures'] = 24
network_params_inv['n_hidden'] = [24]

inverse_model = MDNPushInverseModel(sess=sess, network_params=network_params_inv)
inverse_model.init_model()

In [ ]:
xs = np.random.uniform(-10.5, 10.5, (6, NSAMPLE)).T
ys = np.random.uniform(-10.5, 10.5, (3, NSAMPLE)).T

In [ ]:

r_data = np.float32(np.random.normal(size=(NSAMPLE,1))) # random noise

y_data = np.float32(np.random.uniform(-10.5, 10.5, (1, NSAMPLE))).T
x_data = np.float32(np.sin(0.75*y_data)*7.0+y_data*0.5+r_data*1.0)


plt.figure(figsize=(8, 8))
plt.plot(x_data,y_data,'ro', alpha=0.3)
plt.show()

In [ ]:
sess.run(tf.global_variables_initializer())

NEPOCH = 10000
loss = np.zeros(NEPOCH) # store the training progress here.
for i in range(NEPOCH):
  sess.run(inverse_model._net_ops['train'],feed_dict={inverse_model._net_ops['x']: x_data, inverse_model._net_ops['y']: y_data})
  loss[i] = sess.run(inverse_model._net_ops['loss'], feed_dict={inverse_model._net_ops['x']: x_data, inverse_model._net_ops['y']: y_data})

In [ ]:
print loss

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(np.arange(100, NEPOCH,1), loss[100:], 'r-')
plt.show()

In [ ]:
from aml_dl.mdn.model.tf_mdn_model import MixtureOfGaussians

In [ ]:
out_pi = inverse_model._net_ops['pi']
out_sigma = inverse_model._net_ops['sigma']
out_mu = inverse_model._net_ops['mu']
x = inverse_model._net_ops['x']

In [ ]:
mixture = MixtureOfGaussians()

In [ ]:
x_test = np.float32(np.arange(-15,15,0.1))
NTEST = x_test.size
x_test = x_test.reshape(NTEST,1) # needs to be a matrix, not a vector


In [ ]:
out_pi_test, out_sigma_test, out_mu_test = sess.run([out_pi, out_sigma, out_mu], feed_dict={x: x_test})

In [ ]:
# print out_mu_test.reshape((300,2))

In [ ]:
out_mu_test = out_mu_test.reshape((300,24))
print out_pi_test.shape, x_test.shape, out_sigma_test.shape, out_mu_test.shape


In [ ]:


y_test = mixture.generate_mixture_samples(out_pi_test, out_mu_test, out_sigma_test,100)

plt.figure(figsize=(8, 8))
plt.plot(x_data,y_data,'ro', x_test,y_test,'bo',alpha=0.3)
plt.show()